In [1]:
import requests
import pandas as pd
from collections import defaultdict

## Getting a key to the Guardian API

In [2]:
API_KEY = "408cea44-7deb-4ad1-a1a4-bbe559b52e56"

## Search function and a URL creator

In [3]:
def search_articles(api_key, search_term='', page=1, page_size=100):
    search_term = search_term.replace(' ' , '%20')
    url = 'https://content.guardianapis.com/search'
    params = {
        'api-key': api_key,
        'format': 'json',
        'page': page,
        'page-size': page_size,
        'q': search_term
    }
    response = requests.get(url, params=params)
    return response.json()

In [4]:
def create_urls(search_results):
    urls = []
    for result in search_results:
        api_url = result['apiUrl']
        urls.append(api_url + '?show-fields=body&api-key=408cea44-7deb-4ad1-a1a4-bbe559b52e56')
    return urls

## Preparing the search terms 

In [5]:
search_terms = ['donald trump', 'joe biden', 'ukraine', 'economy', 'israel', 'cryptocurrency', 'immigration']
texts_dict = defaultdict(list)
texts = []

In [6]:
from bs4 import BeautifulSoup
for term in search_terms:
    search_results = search_articles(API_KEY, search_term=term)['response']['results']
    urls = create_urls(search_results)
    for url in urls:
        response = requests.get(url)
        response_json = response.json()
        try:
            response_body = response_json['response']['content']['fields']['body']
            soup = BeautifulSoup(response_body, 'html.parser')
            text = soup.get_text(separator=' ')
            texts.append(text)
            texts_dict[term].append(text)
        except:
            pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [24]:
import pickle
def calculate_article_lengths(texts):
    lengths = {}
    for term, articles in texts.items():
        lengths[term] = [len(text.split()) for text in articles]
    return lengths

article_lengths = calculate_article_lengths(texts_dict)
with open('article_lengths.pkl', 'wb') as file:
    pickle.dump(texts, file)

In [22]:
data_frame = pd.DataFrame(texts, columns=['Texts'])
data_frame

,Texts
0,Donald Trump continued his march toward the GO...
1,"Donald Payne Jr, a US congressman from New Jer..."
2,Donald Trump has been jubilant over the US su...
3,Shortly after the Biden-Harris re-election cam...
4,"The New York City mayor, Eric Adams, said the ..."
...,...
695,We seem to be living in times of unprecedented...
696,9.00pm GMT Closing summary A bill...
697,The UK’s headline immigration policy faces a c...
698,Labour must not make the mistake of being too ...


## Importing libraries for text preprocessing and cleaning

In [15]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mwojc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Cleaning the text and adding a cleaned_text column to the dataframe

In [17]:
lemmatizer = WordNetLemmatizer()

def clean_and_lemmatize(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [18]:
data_frame['cleaned_text'] = data_frame['Texts'].apply(clean_and_lemmatize)

In [19]:
data_frame['cleaned_text']

0      donald trump continued his march toward the go...
1      donald payne jr a u congressman from new jerse...
2      donald trump ha been jubilant over the u supre...
3      shortly after the bidenharris reelection campa...
4      the new york city mayor eric adam said the cit...
                             ...                        
695    we seem to be living in time of unprecedented ...
696    900pm gmt closing summary a bill to enact hard...
697    the uk headline immigration policy face a crun...
698    labour must not make the mistake of being too ...
699    donald trump bemoaned a lack of immigrant to t...
Name: cleaned_text, Length: 700, dtype: object

## Saving the dataframe and displaying a sample preprocessed text

In [20]:
data_frame_cleaned = pd.DataFrame(data_frame, columns=['Texts', 'cleaned_text'])
data_frame_cleaned.to_csv('article_data_cleaned_2.csv', index=False, encoding='utf-8')

In [21]:
data_frame_cleaned['cleaned_text'][0]

'donald trump continued his march toward the gop nomination at the michigan republican party convention on saturday sweeping all 39 delegate related a farright u youth group is ramping up it movement to back election denier the delegate awarded will fuel the former president ahead of tuesday 5 march when 15 state will hold primary and trump nomination could be all but decided the michigan state party delegate met on saturday at the sprawling amway plaza hotel in grand rapid huddling in 13 separate meeting room representing the state 13 congressional district their nearuniform support for trump at the convention eclipsed the support he earned in the primary when former un ambassador nikki haley garnered about 26 of the vote she did not win any delegate awarded on saturday for the republican national convention in milwaukee where the party in july will officially nominate a candidate for the november presidential election the michigan republican party process for awarding delegate to the